In [12]:
import pandas as pd
import csv
import pprint as pprint
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np
#import sklearn.datasets as dta
#from scipy.stats import linregress
#from scipy.stats import sem
import json
import requests
import os
from dotenv import load_dotenv
import gzip
# Load .env enviroment variables into the notebook
load_dotenv()
# Get the API key from the environment variable and store as Python variable
weather_api_key = os.getenv("weather_api_key")

In [13]:
filepath_redfin = "../data_sources/zip_code_market_tracker.tsv000"
redfinraw = pd.read_csv(filepath_redfin, delimiter = "\t")
redfinraw.head()

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
0,2014-08-01,2014-10-31,90,zip code,2,1920,f,Zip Code: 05489,NaN,Vermont,...,-0.325000,NaN,NaN,NaN,NaN,NaN,NaN,"Burlington, VT",15540,2024-03-10 14:36:40
1,2018-04-01,2018-06-30,90,zip code,2,18317,f,Zip Code: 43750,NaN,Ohio,...,NaN,NaN,NaN,NaN,0.0,NaN,NaN,"Cambridge, OH",15740,2024-03-10 14:36:40
2,2017-01-01,2017-03-31,90,zip code,2,13955,f,Zip Code: 33062,NaN,Florida,...,-0.600000,NaN,NaN,NaN,0.0,NaN,-0.500000,"Fort Lauderdale, FL",22744,2024-03-10 14:36:40
3,2015-12-01,2016-02-29,90,zip code,2,23759,f,Zip Code: 55437,NaN,Minnesota,...,-0.181818,NaN,NaN,NaN,0.0,-1.0,-0.166667,"Minneapolis, MN",33460,2024-03-10 14:36:40
4,2023-12-01,2024-02-29,90,zip code,2,25730,f,Zip Code: 59922,NaN,Montana,...,0.000000,NaN,NaN,NaN,1.0,1.0,1.000000,"Kalispell, MT",28060,2024-03-10 14:36:40


In [14]:
cols = list(redfinraw.columns)
cols

['period_begin',
 'period_end',
 'period_duration',
 'region_type',
 'region_type_id',
 'table_id',
 'is_seasonally_adjusted',
 'region',
 'city',
 'state',
 'state_code',
 'property_type',
 'property_type_id',
 'median_sale_price',
 'median_sale_price_mom',
 'median_sale_price_yoy',
 'median_list_price',
 'median_list_price_mom',
 'median_list_price_yoy',
 'median_ppsf',
 'median_ppsf_mom',
 'median_ppsf_yoy',
 'median_list_ppsf',
 'median_list_ppsf_mom',
 'median_list_ppsf_yoy',
 'homes_sold',
 'homes_sold_mom',
 'homes_sold_yoy',
 'pending_sales',
 'pending_sales_mom',
 'pending_sales_yoy',
 'new_listings',
 'new_listings_mom',
 'new_listings_yoy',
 'inventory',
 'inventory_mom',
 'inventory_yoy',
 'months_of_supply',
 'months_of_supply_mom',
 'months_of_supply_yoy',
 'median_dom',
 'median_dom_mom',
 'median_dom_yoy',
 'avg_sale_to_list',
 'avg_sale_to_list_mom',
 'avg_sale_to_list_yoy',
 'sold_above_list',
 'sold_above_list_mom',
 'sold_above_list_yoy',
 'price_drops',
 'price_dro

In [24]:
allzipcodes = [
98102,
98103,
98107,
98109,
98112,
98115,
98117,
98119,
98125,
98133,
98177,
98105,
98106,
98108,
98112,
98116,
98118,
98122,
98126,
98134,
98136,
98144,
98146,
98178,
98199,
98101,
98104,
98113,
98121,
98141,
98154,
98161,
98165,
98170,
98174,
98175,
98181,
98185,
98190,
98191,
98194
]
# allzipcodes_str = [str(zipcode) for zipcode in allzipcodes]


In [25]:
allzipcodes_format = []

# Iterate over each zip code, convert to string, and append to the new list with the "Zip Code:" prefix
for zipcode in allzipcodes:
    modified_zipcode = "Zip Code: " + str(zipcode)  # Convert to string and prepend "Zip Code: "
    allzipcodes_format.append(modified_zipcode)

print(allzipcodes_format)

['Zip Code: 98102', 'Zip Code: 98103', 'Zip Code: 98107', 'Zip Code: 98109', 'Zip Code: 98112', 'Zip Code: 98115', 'Zip Code: 98117', 'Zip Code: 98119', 'Zip Code: 98125', 'Zip Code: 98133', 'Zip Code: 98177', 'Zip Code: 98105', 'Zip Code: 98106', 'Zip Code: 98108', 'Zip Code: 98112', 'Zip Code: 98116', 'Zip Code: 98118', 'Zip Code: 98122', 'Zip Code: 98126', 'Zip Code: 98134', 'Zip Code: 98136', 'Zip Code: 98144', 'Zip Code: 98146', 'Zip Code: 98178', 'Zip Code: 98199', 'Zip Code: 98101', 'Zip Code: 98104', 'Zip Code: 98113', 'Zip Code: 98121', 'Zip Code: 98141', 'Zip Code: 98154', 'Zip Code: 98161', 'Zip Code: 98165', 'Zip Code: 98170', 'Zip Code: 98174', 'Zip Code: 98175', 'Zip Code: 98181', 'Zip Code: 98185', 'Zip Code: 98190', 'Zip Code: 98191', 'Zip Code: 98194']


In [27]:
metrodata = redfinraw[redfinraw["region"].isin(allzipcodes_format)]
metrodata

,period_begin,period_end,period_duration,region_type,region_type_id,table_id,is_seasonally_adjusted,region,city,state,...,sold_above_list_yoy,price_drops,price_drops_mom,price_drops_yoy,off_market_in_two_weeks,off_market_in_two_weeks_mom,off_market_in_two_weeks_yoy,parent_metro_region,parent_metro_region_metro_code,last_updated
71,2018-03-01,2018-05-31,90,zip code,2,40833,f,Zip Code: 98121,NaN,Washington,...,-0.108458,NaN,NaN,NaN,0.777778,-0.011696,-0.116959,"Seattle, WA",42644,2024-03-10 14:36:40
1907,2014-05-01,2014-07-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.004266,NaN,NaN,NaN,0.712121,-0.129984,0.109971,"Seattle, WA",42644,2024-03-10 14:36:40
3522,2014-09-01,2014-11-30,90,zip code,2,40865,f,Zip Code: 98175,NaN,Washington,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4375,2013-11-01,2014-01-31,90,zip code,2,40816,f,Zip Code: 98103,NaN,Washington,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,"Seattle, WA",42644,2024-03-10 14:36:40
4781,2018-09-01,2018-11-30,90,zip code,2,40834,f,Zip Code: 98122,NaN,Washington,...,-0.411765,NaN,NaN,NaN,0.333333,0.183333,-0.066667,"Seattle, WA",42644,2024-03-10 14:36:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,zip code,2,40821,f,Zip Code: 98108,NaN,Washington,...,-0.083333,NaN,NaN,NaN,0.800000,0.400000,0.466667,"Seattle, WA",42644,2024-03-10 14:36:40
7842141,2016-03-01,2016-05-31,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.233333,NaN,NaN,NaN,0.500000,-0.500000,-0.500000,"Seattle, WA",42644,2024-03-10 14:36:40
7842261,2022-03-01,2022-05-31,90,zip code,2,40825,f,Zip Code: 98112,NaN,Washington,...,0.293651,NaN,NaN,NaN,0.888889,0.111111,-0.111111,"Seattle, WA",42644,2024-03-10 14:36:40
7842290,2015-07-01,2015-09-30,90,zip code,2,40850,f,Zip Code: 98146,NaN,Washington,...,0.666667,NaN,NaN,NaN,0.500000,-0.166667,NaN,"Seattle, WA",42644,2024-03-10 14:36:40


In [43]:
metrodata2 = metrodata[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
# metrodata2.reset_index().sort_values(by="period_begin", ascending = True)
metrodata2.reset_index().head()
metrodata2 = metrodata2[[
    'period_begin',
    'period_end',
    'period_duration',
    'table_id',
    'region',
    'city',
    'state',
    'state_code',
    'property_type',
    'property_type_id',
    'median_sale_price',
    'median_sale_price_yoy',
    'median_list_price',
    'median_list_price_yoy',
    'median_ppsf',
    'median_ppsf_yoy',
    'median_list_ppsf',
    'median_list_ppsf_yoy',
    'homes_sold',
    'homes_sold_yoy',
    'pending_sales',
    'pending_sales_yoy',
    'new_listings',
    'new_listings_yoy',
    'inventory',
    'inventory_mom',
    'inventory_yoy',
    'months_of_supply',
    'months_of_supply_yoy',
    'avg_sale_to_list',
    'avg_sale_to_list_yoy',
    'sold_above_list',
    'sold_above_list_yoy',
    'price_drops',
    'price_drops_yoy',
    'parent_metro_region']]
metrodata2

,period_begin,period_end,period_duration,table_id,region,city,state,state_code,property_type,property_type_id,...,inventory_yoy,months_of_supply,months_of_supply_yoy,avg_sale_to_list,avg_sale_to_list_yoy,sold_above_list,sold_above_list_yoy,price_drops,price_drops_yoy,parent_metro_region
71,2018-03-01,2018-05-31,90,40833,Zip Code: 98121,NaN,Washington,WA,All Residential,-1,...,1.476190,NaN,NaN,1.037550,-0.010136,0.520408,-0.108458,NaN,NaN,"Seattle, WA"
1907,2014-05-01,2014-07-31,90,40816,Zip Code: 98103,NaN,Washington,WA,All Residential,-1,...,0.034884,NaN,NaN,1.030420,0.000531,0.545082,0.004266,NaN,NaN,"Seattle, WA"
3522,2014-09-01,2014-11-30,90,40865,Zip Code: 98175,NaN,Washington,WA,Townhouse,13,...,NaN,NaN,NaN,0.986301,NaN,0.000000,NaN,NaN,NaN,"Seattle, WA"
4375,2013-11-01,2014-01-31,90,40816,Zip Code: 98103,NaN,Washington,WA,Multi-Family (2-4 Unit),4,...,0.333333,NaN,NaN,0.969471,0.002038,0.000000,0.000000,NaN,NaN,"Seattle, WA"
4781,2018-09-01,2018-11-30,90,40834,Zip Code: 98122,NaN,Washington,WA,Townhouse,13,...,5.000000,NaN,NaN,0.989249,-0.036230,0.088235,-0.411765,NaN,NaN,"Seattle, WA"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7841828,2019-12-01,2020-02-29,90,40821,Zip Code: 98108,NaN,Washington,WA,Townhouse,13,...,-0.333333,NaN,NaN,0.990874,0.008804,0.000000,-0.083333,NaN,NaN,"Seattle, WA"
7842141,2016-03-01,2016-05-31,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,1.500000,NaN,NaN,1.007034,0.023603,0.400000,0.233333,NaN,NaN,"Seattle, WA"
7842261,2022-03-01,2022-05-31,90,40825,Zip Code: 98112,NaN,Washington,WA,Condo/Co-op,3,...,-0.636364,NaN,NaN,1.033408,0.025816,0.571429,0.293651,NaN,NaN,"Seattle, WA"
7842290,2015-07-01,2015-09-30,90,40850,Zip Code: 98146,NaN,Washington,WA,Condo/Co-op,3,...,-0.500000,NaN,NaN,1.026968,0.052054,0.666667,0.666667,NaN,NaN,"Seattle, WA"


In [39]:
groupedbypropertytype = metrodata2.groupby("property_type")
groupedbypropertytype

In [9]:
# metrodata = metrodata[
#     'period_begin',
#     'period_end',
#     'period_duration',
#     'region_type',
#     'region_type_id',
#     'table_id',
#     'is_seasonally_adjusted',
#     'region',
#     'city',
#     'state',
#     'state_code',
#     'property_type',
#     'property_type_id',
#     'median_sale_price',
#     'median_sale_price_mom',
#     'median_sale_price_yoy',
#     'median_list_price',
#     'median_list_price_mom',
#     'median_list_price_yoy',
#     'median_ppsf',
#     'median_ppsf_mom',
#     'median_ppsf_yoy',
#     'median_list_ppsf',
#     'median_list_ppsf_mom',
#     'median_list_ppsf_yoy',
#     'homes_sold',
#     'homes_sold_mom',
#     'homes_sold_yoy',
#     'pending_sales',
#     'pending_sales_mom',
#     'pending_sales_yoy',
#     'new_listings',
#     'new_listings_mom',
#     'new_listings_yoy',
#     'inventory',
#     'inventory_mom',
#     'inventory_yoy',
#     'months_of_supply',
#     'months_of_supply_mom',
#     'months_of_supply_yoy',
#     'median_dom',
#     'median_dom_mom',
#     'median_dom_yoy',
#     'avg_sale_to_list',
#     'avg_sale_to_list_mom',
#     'avg_sale_to_list_yoy',
#     'sold_above_list',
#     'sold_above_list_mom',
#     'sold_above_list_yoy',
#     'price_drops',
#     'price_drops_mom',
#     'price_drops_yoy',
#     'off_market_in_two_weeks',
#     'off_market_in_two_weeks_mom',
#     'off_market_in_two_weeks_yoy',
#     'parent_metro_region',
#     'parent_metro_region_metro_code',
#     'last_updated']

In [44]:
metrodata2.to_csv("../data_sources/redfin_by_zipcode.csv", index=False)